## 복습 프로세스
- 네이버 뉴스 검색 제목 크롤링
- 네이버 뉴스 크롤링
- 네이버 자동 검색
- 네이버 연관 검색어 3차 크롤링(엑셀)
- 유튜브 랭킹 크롤링
- 판다스 이름 분석
- 선별진료소 지도

### 준비

In [2]:
# 라이브러리
import chromedriver_autoinstaller
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from tqdm import tqdm
from datetime import datetime

import folium
from folium.plugins import MarkerCluster
from folium.plugins import MiniMap

# chromedriver 버전 업데이트
chromedriver_autoinstaller.install()

'C:\\Users\\Drimsys\\anaconda3\\lib\\site-packages\\chromedriver_autoinstaller\\96\\chromedriver.exe'

### 네이버 뉴스 검색 제목 크롤링

In [2]:
# 브라우저 열기
browser = webdriver.Chrome()

# 웹페이지 접속
url = "http://www.naver.com"
browser.get(url)

time.sleep(1)

# 찍기
html=browser.page_source
soup=BeautifulSoup(html, "html.parser")

In [3]:
# 테스트
title=soup.select("a.news_tit")
len(title)

0

In [4]:
# 검색 리스트
search_word_list=["파이썬", "자바"]

# 크롤링 진행
# 검색 리스트 반복
for search_word in search_word_list:
    # 검색 리스트 페이지 열기
    url=f"https://search.naver.com/search.naver?where=news&sm=tab_jum&query={search_word}"
    browser.get(url)
    
    # 찍기
    html=browser.page_source
    soup=BeautifulSoup(html, "html.parser")
    
    time.sleep(1)
    
    # 제목 태그
    title=soup.select("a.news_tit")[0].text
    print(title.strip())

성안당, 개발자용 고급 파이썬 입문서 '프로그래머를 위한 Python' 출간
구글, 취약한 로그4j 사용하는 자바 패키지 35863개 찾아


### 네이버 뉴스 크롤링

In [5]:
# 브라우저 열기
browser = webdriver.Chrome()

# 웹페이지 접속
url = "https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=031&aid=0000637022"
browser.get(url)

time.sleep(1)

# 찍기
html=browser.page_source
soup=BeautifulSoup(html, "html.parser")

In [6]:
# 테스트
title=soup.select("h3#articleTitle")
date=soup.select("span.t11")
press=soup.select("div.press_logo > a > img")
len(press)

1

In [7]:
# 속성값 테스트
soup.select("div.press_logo > a > img")[0]["title"]

'아이뉴스24'

In [8]:
# 브라우저 열기
browser = webdriver.Chrome()

# 웹페이지 접속
url = "https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=031&aid=0000637022"
browser.get(url)

time.sleep(1)

# 찍기
html=browser.page_source
soup=BeautifulSoup(html, "html.parser")

# 태그
title=soup.select("h3#articleTitle")[0].text.strip()
date=soup.select("span.t11")[0].text.strip()
press=soup.select("div.press_logo > a > img")[0]["title"].strip()

# 출력
print(title)
print(date)
print(press)

승현준 삼성리서치 소장 "개방성과 협업이 소프트웨어의 핵심"
2021.11.18. 오전 10:01
아이뉴스24


### 네이버 자동 검색

In [3]:
# 브라우저 열기
browser = webdriver.Chrome()

# 웹페이지 접속
url="http://www.naver.com"
browser.get(url)

word_list=["파이썬", "자바"]
for word in tqdm(word_list):
    # 검색 단어 입력
    searching=browser.find_elements_by_css_selector("input#query")[0]
    searching.clear()
    searching.send_keys(word)

    # 검색
    search_btn=browser.find_elements_by_css_selector("button#search_btn")[0]
    search_btn.click()
    
    # 되돌아가기
    browser.back()

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]<ipython-input-3-2a0fd1809f64>:11: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  searching=browser.find_elements_by_css_selector("input#query")[0]
<ipython-input-3-2a0fd1809f64>:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  search_btn=browser.find_elements_by_css_selector("button#search_btn")[0]
100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.92s/it]


### 네이버 연관 검색어 3차 크롤링(엑셀)

In [10]:
# excel 파일 열기
raw=pd.read_excel("./data/searching_word.xlsx")
word_list=raw["검색어"]
word_list

0    파이썬
1     자바
Name: 검색어, dtype: object

In [11]:
# 날짜 테스트
now=datetime.now().isoformat()[:10]
now

'2021-12-23'

In [12]:
# 브라우저 열기
browser = webdriver.Chrome()

# 웹페이지 접속
url="http://www.naver.com"
browser.get(url)

# 검색 단어 입력
searching=browser.find_elements_by_css_selector("input#query")[0]
searching.clear()
searching.send_keys(word_list[0])

time.sleep(1)

# 찍기
html=browser.page_source
soup=BeautifulSoup(html, "html.parser")

# 테스트
related_word=soup.select("ul.kwd_lst._kwd_list > li.item._item > a > span.fix")
len(related_word)

<ipython-input-12-f041bcb89542>:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  searching=browser.find_elements_by_css_selector("input#query")[0]


10

In [13]:
# excel 파일로 저장 함수
def save_excel(result):
    df=pd.DataFrame(results)
    df.columns=["검색어", "연관검색어", "검색일시"]
    df.to_excel("./result/Result_Searching_1.xlsx", index=False)
    return df

In [14]:
# 브라우저 열기
browser = webdriver.Chrome()

# 웹페이지 접속
url="http://www.naver.com"
browser.get(url)

word=word_list[0]
results=[]

# 검색 단어 입력
searching=browser.find_elements_by_css_selector("input#query")[0]
searching.clear()
searching.send_keys(word_list[0])

# 검색 시간
now=datetime.now().isoformat()[:10]

time.sleep(1)

# 찍기
html=browser.page_source
soup=BeautifulSoup(html, "html.parser")

# 연관 검색어 태그
for tag in soup.select("ul.kwd_lst._kwd_list > li.item._item > a > span.fix"):
    related_word=tag.text
    if related_word != word:
        results.append([word, related_word, now])
df=save_excel(results)
df.head()

<ipython-input-14-a41eca74fb2f>:12: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  searching=browser.find_elements_by_css_selector("input#query")[0]


,검색어,연관검색어,검색일시
0,파이썬,파이썬 자격증,2021-12-23
1,파이썬,파이썬 독학,2021-12-23
2,파이썬,파이썬 리스트,2021-12-23
3,파이썬,파이썬 설치,2021-12-23
4,파이썬,파이썬 함수,2021-12-23


### 유튜브 랭킹 크롤링

In [15]:
# 브라우저 열기
browser = webdriver.Chrome() # 컴퓨터용 브라우저

url="https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1"
browser.get(url)

time.sleep(1)

# 100개 채널, 채널명, 카테고리, 구독자수, 조회수, 영상 개수
html=browser.page_source
soup=BeautifulSoup(html, "html.parser")

### 판다스 이름 분석

In [16]:
raw=pd.read_csv("./data/babyNamesUS.csv")
raw.head()

,StateCode,Sex,YearOfBirth,Name,Number
0,AK,F,1910,Mary,14
1,AK,F,1910,Annie,12
2,AK,F,1910,Anna,10
3,AK,F,1910,Margaret,8
4,AK,F,1910,Helen,7


### 선별진료소 지도

In [17]:
# 테스트
# 서울역 위도, 경도
seoul_station = [37.5536387,126.9671714]
map=folium.Map(location=seoul_station, zoom_start=12)

# 마커하기
folium.Marker(seoul_station, tooltip="Seoul", popup="Station").add_to(map)
map

In [18]:
raw=pd.read_excel("./data/선별진료소.xlsx")
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626 entries, 0 to 625
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   시도            626 non-null    object 
 1   시군구           626 non-null    object 
 2   의료기관명         626 non-null    object 
 3   주소            626 non-null    object 
 4   평일 운영시간       626 non-null    object 
 5   토요일 운영시간      626 non-null    object 
 6   일요일/공휴일
운영시간  626 non-null    object 
 7   대표 전화번호       626 non-null    object 
 8   관할보건소         626 non-null    object 
 9   관할보건소 전화번호    626 non-null    object 
 10  place_name    626 non-null    object 
 11  type          626 non-null    object 
 12  x             626 non-null    float64
 13  y             626 non-null    float64
dtypes: float64(2), object(12)
memory usage: 68.6+ KB


In [19]:
lat=raw.loc[1, "y"] # 위도 : 남북
long=raw.loc[1, "x"] # 경도 : 동서
name=raw.loc[1, "place_name"]
call=raw.loc[1, "대표 전화번호"]
print(lat, long, name, call)

37.4880999767092 127.085373632816 삼성서울병원 02-1599-3114


In [20]:
raw.head(1)

,시도,시군구,의료기관명,주소,평일 운영시간,토요일 운영시간,일요일/공휴일\n운영시간,대표 전화번호,관할보건소,관할보건소 전화번호,place_name,type,x,y
0,서울,강남구,강남구보건소,서울 강남구 삼성동(삼성2동) 8 강남구보건소,09:00 ~ 19:00,09:00 ~ 17:00,09:00 ~ 17:00,02-3423-5555,강남구보건소,02-3423-7138,강남구보건소,선별진료소,127.042314,37.516321


In [21]:
# 서울역 위도, 경도
seoul_station = [37.5536387,126.9671714]
map=folium.Map(location=seoul_station, zoom_start=12)

raw=pd.read_excel("./data/선별진료소.xlsx")

# 미니맵과 클러스터
MiniMap().add_to(map)
marker_cluster=MarkerCluster().add_to(map)

# 마커하기
for i in raw.index:
    lat=raw.loc[i, "y"] # 위도 : 남북
    long=raw.loc[i, "x"] # 경도 : 동서
    name=raw.loc[i, "place_name"]
    call=raw.loc[i, "대표 전화번호"]

    location=[lat, long]

    folium.Marker(location, tooltip=name, 
                  popup=call).add_to(marker_cluster)
map

In [22]:
map.save("./result/COVID_Clinic_GEO.html")